In [1]:
from EmotionAnalysis.SentSemanticModule import *
from EmotionAnalysis.SentTweetModule import *
import pandas as pd

## I. Training Word2Vec Model for Multiple Languages:

### 1. Loading Tokenized Lemmatized Representation before further cleaning:

In [ ]:
# LOADING TOKENIZED REPRESENTATION
tokenized = pd.read_csv('../../Project_Backup/BigData/Unannotated_Representation/fr/Unannotated_Representation1_1.csv',encoding="ISO-8859-1")
tokenized1 = pd.read_csv('../../Project_Backup/BigData/Unannotated_Representation/fr/Unannotated_Representation1_2.csv',encoding="ISO-8859-1")
tokenized2 = pd.read_csv('../../Project_Backup/BigData/Unannotated_Representation/fr/Unannotated_Representation1_3.csv',encoding="ISO-8859-1")
tokenized3 = pd.read_csv('../../Project_Backup/BigData/Unannotated_Representation/fr/Unannotated_Representation1_4.csv',encoding="ISO-8859-1")
tokenized4 = pd.read_csv('../../Project_Backup/BigData/Unannotated_Representation/fr/Unannotated_Representation1_5.csv',encoding="ISO-8859-1")
tokenized5 = pd.read_csv('../../Project_Backup/BigData/Unannotated_Representation/fr/Unannotated_Representation1_sw.csv',encoding="ISO-8859-1")
tokenized6 = pd.read_csv('../../Project_Backup/BigData/Unannotated_Representation/fr/Unannotated_Representation2_sw.csv',encoding="ISO-8859-1")

#from tqdm import tqdm
#import ast
tokenized_lemma = list(tokenized['Tokenized Lemmatization'])+list(tokenized1['Tokenized Lemmatization'])+list(tokenized2['Tokenized Lemmatization'])+list(tokenized3['Tokenized Lemmatization'])+list(tokenized4['Tokenized Lemmatization'])+list(tokenized5['Tokenized Lemmatized'])+list(tokenized6['Tokenized Lemmatized'])
tokenized_lemmatized_tweets = []
for i in tqdm(range(0, len(tokenized_lemma))):
    #result = ast.literal_eval(tokenized_lemma[i])
    #tokenized_lemmatized_tweets.append(result)
    tokenized_lemmatized_tweets.append(tokenized_lemma[i][1:len(tokenized_lemma[i])-1].split(', '))


### 2. Training and Fine-Tuning Word Embedding Model using gensim word2vec: 

In [ ]:
# PARAMETERS TO BE TUNED:

# Word vector dimensionality                      
# Minimum word count                        
# Number of threads to run in parallel
# Context window size                                                                                    
# Downsample setting for frequent words

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality
min_word_count = 4  # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(tokenized_lemmatized_tweets, workers=num_workers,
            size=num_features, min_count = min_word_count,
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "../../Project_Backup/BigData/Models/whole_fr_model"
model.save(model_name)

## II. Applying Rule Based Lexicon Approach Extended with Word2Vec Methodology:

### 1. Loading Tweets in their Affective Representation form:

In [13]:
import pandas as pd
tokenized = pd.read_csv('../../Project_Backup/BigData/Unannotated_Representation/fr/Unannotated_Representation1_5.csv',encoding="ISO-8859-1")
#tokenized1 = pd.read_csv('../../Project_Backup/BigData/Unannotated_Representation/de/Unannotated_Representation2_sw.csv',encoding="ISO-8859-1")
#tokenized2 = pd.read_csv('../../Project_Backup/BigData/Unannotated_Representation/de/Unannotated_Representation3_sw.csv',encoding="ISO-8859-1")

from tqdm import tqdm
import ast

nava_repr = list(tokenized['Nava without Stop Words'])#+list(tokenized1['Nava without Stop Words'])+list(tokenized2['Nava without Stop Words'])
nava_repr[0]

'[demain, être, anniversaire, avoir, enfin, an]'

In [14]:
print ("Convert nava tweets")
# Convert nava_tweets 
nava_tweets = []
for i in tqdm(range(0, len(nava_repr))):
    #result = ast.literal_eval(nava_repr[i])
    #nava_tweets.append(result)
    nava_tweets.append(nava_repr[i][1:len(nava_repr[i])-1].split(', '))
nava_tweets[1]

Convert nava tweets


100%|██████████████████████████████| 500272/500272 [00:01<00:00, 441022.68it/s]


['jonathan',
 'toews',
 'être',
 'titres',
 'olympique',
 'coupe',
 'stanley',
 'championnat',
 'monde',
 'seulement',
 'an',
 'nhl']

### 2. Loading Language Lexicon (version containing set of representative words): 

In [4]:
lexicon_df = pd.read_csv('NRCLexicon/French/french_nrc.csv',encoding='ISO-8859-1')

### 3. Loading Word2Vec Model (if it was already Pre-trained):

In [5]:
print ("Loading Word2Vec ....")
from gensim.models import word2vec
model = word2vec.Word2Vec.load('../../Project_Backup/BigData/Models/whole_fr_model')

Loading Word2Vec ....


C:\Users\fnac\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
def compute_matrix_sentences_list_word2vec(nava_words, nrc_lexicon,model):
    """

    :param word2vec model:
    :param nava_words: we can pass any version of the bag of words
    :param nrc_lexicon:
    :return:
    """

    sm_list = list_nrc_lexicon(nrc_lexicon)
    emotions = nrc_lexicon.columns.values
    matrix_sentences_list = []
    for i in tqdm(range(0, len(nava_words))): # Iterate over all sentences
        " Initialize matrix for each sentence "
        w, h = len(nava_words[i]), 10
        matrix_sentence = [[0 for x in range(w)] for y in range(h)]
        k = 0
        for word in nava_words[i]: # Iterate over all words in the sentence
            j = 0
            for emotion in range(0, len(emotions)): # Iterate over all emotions => fill in the emotional vectors for all words
                total_similarity = 0
                for representative_word in sm_list[emotion][0:10]:
                    r = len(sm_list[emotion])
                    if word in model and representative_word in model:
                        total_similarity += model.similarity(word, representative_word)
                matrix_sentence[j][k] += total_similarity / r 
                j += 1 # increment index of representative words
            k += 1 # increment index of transcript words
        # append the matrix_sentence to the global list for all sentences
        matrix_sentences_list.append(matrix_sentence)
    return matrix_sentences_list

### 4. Computing Word Level Emotional Vectors:

In [15]:
###### STEP 4: Word Level
print ("Computing word level scores")
matrix_sentences_word2vec = compute_matrix_sentences_list_word2vec(nava_tweets,lexicon_df,model)

Computing word level scores


100%|█████████████████████████████████| 500272/500272 [42:59<00:00, 193.92it/s]


In [8]:
from __future__ import division

def compute_sentence_emotion_vectors(matrix_sentences_list):
    emotion_vector_list = []
    for i in tqdm(range(0, len(matrix_sentences_list))):
        sum_sentence = []
        ids = [0,1,2,3,4,7,8,9]
        for j in ids: # for each emotion
            sum_words = 0
            for k in range(0, len(matrix_sentences_list[i][j])):
                sum_words += matrix_sentences_list[i][j][k]*1000
            r = len(matrix_sentences_list[i])
            if r != 0 :
                sum_words = sum_words / r # Arithmetic mean
            sum_sentence.append(sum_words)
        emotion_vector_list.append(sum_sentence)
    return emotion_vector_list

def compute_sentence_sentiment_vectors(matrix_sentences_list):
    emotion_vector_list = []
    for i in tqdm(range(0, len(matrix_sentences_list))):
        sum_sentence = []
        ids = [5,6]
        for j in ids: # for each emotion
            sum_words = 0
            for k in range(0, len(matrix_sentences_list[i][j])):
                sum_words += matrix_sentences_list[i][j][k]*1000
            r = len(matrix_sentences_list[i])
            if r != 0 : 
                sum_words = sum_words / r
            sum_sentence.append(sum_words)
        emotion_vector_list.append(sum_sentence)
    return emotion_vector_list

def compute_emotionalities(sentence_vectors):
    emotionalities = []
    threshold = 0 # THRESHOLD PARAMETER TO BE FINE TUNED (0 for lexicon, 0.2 for pmi)
    for i in tqdm(range(0,len(sentence_vectors))):
        sentence_vector = sentence_vectors[i]
        mylist = [0 if math.isnan(x) else x for x in sentence_vector]
        if (max(mylist) > threshold): #Threshold 
            emotionalities.append(sentence_vectors[i].index(max(mylist)))
        else: 
            emotionalities.append(8)
    return emotionalities

def compute_sentiments(sentence_vectors_sent,emotionalities):
    sentiments = []
    threshold = 0 # THRESHOLD PARAMETER TO BE FINE TUNED (0 for lexicon, 0.2 for pmi)
    for i in tqdm(range(0,len(sentence_vectors_sent))):
        sentence_vector = sentence_vectors_sent[i]
        mylist = [0 if math.isnan(x) else x for x in sentence_vector]
        if (max(mylist) > threshold): #Threshold 
            sentiments.append(sentence_vectors_sent[i].index(max(mylist)))
        else:
            # To increase Recall, we also use emotionalities, in case a tweet is neutral
            if emotionalities[i] in [0,2,3,5]:
                sentiments.append(0) # Negative Emotion
            if emotionalities[i] in [1,4,6,7]:
                sentiments.append(1) # Positive Emotion
            if emotionalities[i] == 8:
                sentiments.append(2) # Otherwise, we just return Neutral
    return sentiments

### 5. Computing Tweet Level Emotionalities: 

In [16]:
print ("Computing Emotionalities ...")
# Computing vectors of emotional scores by averaging over the word emotion scores
sentence_vectors_word2vec = compute_sentence_emotion_vectors(matrix_sentences_word2vec)

# Selecting dominant emotion using a specific threshold
emotionalities = compute_emotionalities(sentence_vectors_word2vec)

Computing Emotionalities ...


100%|██████████████████████████████| 500272/500272 [00:01<00:00, 283138.48it/s]


### 6. Computing Tweet Level Sentiments:

In [17]:
print ("Computing sentiments ...")

# Computing vectors of polarity scores by averaging over the word polarity scores
sentence_vectors_sent = compute_sentence_sentiment_vectors(matrix_sentences_word2vec)

# Selecting dominant polarity (positive, negative) using a specific threshold
sentiments = compute_sentiments(sentence_vectors_sent,emotionalities)

Computing sentiments ...


100%|██████████████████████████████| 500272/500272 [00:01<00:00, 490207.46it/s]


In [18]:
###### Storing Emotion + Sentiment for each tweet in two lists to be used for storing in dataframe:
emo_dict = {
    0: 'Anger',
    1: 'Anticipation',
    2: 'Disgust',
    3: 'Fear',
    4: 'Joy',
    5: 'Sadness',
    6: 'Surprise',
    7: 'Trust',
    8: 'Neutral'
}
sent_dict = {
    0: "Negative",
    1: "Positive",
    2: "Neutral"
}

emotions = []
senti = []
for i in range(0,len(emotionalities)):
    emotions.append(emo_dict[emotionalities[i]])
    senti.append(sent_dict[sentiments[i]])

### 7. Storing emotion, sentiment and their score vectors in dataframe:

In [19]:
print ("Storing in dataframe ... ")
word2vec_results_df = pd.DataFrame()

word2vec_results_df['Nava Tweet'] = nava_tweets

word2vec_results_df['Emotion Vectors'] = sentence_vectors_word2vec

word2vec_results_df['Emotion'] = emotions

word2vec_results_df['Sentiment Vectors'] = sentence_vectors_sent

word2vec_results_df['Sentiment'] = senti

word2vec_results_df.to_csv('../../Project_Backup/BigData/Word2VecBasedResults/fr/Tweets_Labelled_Word2Vec7.csv')
word2vec_results_df.head()

Storing in dataframe ... 


,Nava Tweet,Emotion Vectors,Emotion,Sentiment Vectors,Sentiment
0,"[demain, être, anniversaire, avoir, enfin, an]","[0.153952701655, 0.357223911084, 0.13792752622...",Surprise,"[0.0125838372616, 0.0311227938282]",Positive
1,"[jonathan, toews, être, titres, olympique, cou...","[0.463664025889, 1.32469796129, 0.731135593957...",Surprise,"[0.0749773618688, 0.280804676131]",Positive
2,"[arriver, croire, passer, annã, soir, être, va...","[0.261061874631, 0.24749203156, 0.222701023972...",Surprise,"[0.0298613381242, 0.00473977164812]",Negative
3,"[an, taire, togo, train, occuper, enfants, app...","[0.584683232545, 0.724039843894, 0.55437827569...",Surprise,"[0.0949304638546, 0.135519443828]",Positive
4,"[aller, taper, examen, blanc, angler, italien,...","[0.36168571456, 0.499481484067, 0.505751252413...",Surprise,"[0.0911929829087, 0.0953891322856]",Positive
